In [1]:
# import libraries 
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split


In [2]:
# import dataset 
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
print("Path to dataset files:", path)

# List files in the directory to find the correct file name
files = os.listdir(path)
print("Files in directory:", files)
# Assuming the correct file is the first CSV file in the directory
csv_file = os.path.join(path, [file for file in files if file.endswith('.csv')][0])
# Load data into a pandas DataFrame
df = pd.read_csv(csv_file)
# Display the first few rows
df.head()

100%|██████████| 5.20G/5.20G [09:12<00:00, 10.1MB/s]

Extracting files...


Path to dataset files: /Users/davidecosta/.cache/kagglehub/datasets/kmader/skin-cancer-mnist-ham10000/versions/2
Files in directory: ['hmnist_28_28_L.csv', 'hmnist_28_28_RGB.csv', 'HAM10000_images_part_1', 'hmnist_8_8_RGB.csv', 'hmnist_8_8_L.csv', 'HAM10000_images_part_2', 'HAM10000_metadata.csv']


,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel0775,pixel0776,pixel0777,pixel0778,pixel0779,pixel0780,pixel0781,pixel0782,pixel0783,label
0,169,171,170,177,181,182,181,185,194,192,...,184,186,185,180,157,140,140,159,165,2
1,19,57,105,140,149,148,144,155,170,170,...,172,175,160,144,114,89,47,18,18,2
2,155,163,161,167,167,172,155,152,165,175,...,163,178,157,166,167,148,141,136,115,2
3,25,71,116,139,136,153,148,161,172,162,...,125,135,138,137,111,71,32,16,16,2
4,129,162,181,196,205,208,205,213,225,224,...,210,197,172,190,195,193,181,147,88,2
